In [1]:
import pandas as pd
import nltk
import realfunctions
import twitter
import numpy
import pickle

we need to make sure we are using the labels "bot" and "human"

In [25]:
run_var = 0

In [26]:
text_data = pd.read_csv('tweets.csv')

df1 = pd.DataFrame({"Tweet": text_data['text'], "Status": text_data['category']}).truncate(after=50000)

In [27]:
# 2016 Nominees
trump = 'realDonaldTrump'
clinton = 'HillaryClinton'

# Top 6 Prospects
biden = 'JoeBiden'
booker = 'CoryBooker'
sanders = 'BernieSanders'

harris = 'KamalaHarris'
gillibrand = 'SenGillibrand'
warren = 'elizabethforma'

# DataFrames for favorites/retweets/tweets for each politician
trump_data = twitter.tweets_favorites_retweets(trump)
clinton_data = twitter.tweets_favorites_retweets(clinton)

biden_data = twitter.tweets_favorites_retweets(biden)
booker_data = twitter.tweets_favorites_retweets(booker)
sanders_data = twitter.tweets_favorites_retweets(sanders)

harris_data = twitter.tweets_favorites_retweets(harris)
gillibrand_data = twitter.tweets_favorites_retweets(gillibrand)
warren_data = twitter.tweets_favorites_retweets(warren)

tweets = trump_data['tweets']
tweets = tweets.append(clinton_data['tweets'])
tweets = tweets.append(biden_data['tweets'])
tweets = tweets.append(booker_data['tweets'])
tweets = tweets.append(sanders_data['tweets'])
tweets = tweets.append(harris_data['tweets'])
tweets = tweets.append(gillibrand_data['tweets'])
tweets = tweets.append(warren_data['tweets'])

traits = ["Human"] * tweets.size

In [28]:
out = pd.DataFrame({'Tweet': tweets, 'Status': traits})

all = df1.append(out)
all = all.reset_index()
out = out.reset_index()

In [29]:
#all = pd.DataFrame({'Tweet': all['Tweet'], 'Status': all['Status']})
all.loc[23400]


index                                                 23400
Status                                                  Bot
Tweet     RT @The_Trump_Train: The Obama administration ...
Name: 23400, dtype: object

In [30]:
sentiments = realfunctions.polarizeFrameNoDate(all['Tweet'])
all = pd.concat([all, sentiments], axis=1)

In [31]:
all.loc[18002]
#sentiments.loc[20000]

index                                                     18002
Status                                                      Bot
Tweet         RT @brownblaze: I read an interview years ago ...
Compound                                                -0.3182
Negativity                                                0.103
Neutrality                                                0.897
Positivity                                                    0
Tweet         RT @brownblaze: I read an interview years ago ...
Name: 18002, dtype: object

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

feats = pd.DataFrame({'Features': ['tweet', 'numwords', 'pos', 'neg', 'neut', 'comp']})
# Organize our data
label_names = all['Status'].unique()
labels = all['Status'].values
feature_names = feats['Features'].values
alltweets = all['Tweet'].values
toFloat = []
words = []
posit = all['Positivity'].values
negat = all['Negativity'].values
neutr = all['Neutrality'].values
compo = all['Compound'].values
for i in range(len(alltweets)):
    toFloat.append(str(alltweets[i]))
    words.append(len(str(alltweets[i]).split()))
    

le = preprocessing.LabelEncoder()
le.fit(toFloat)
toFloat = le.transform(toFloat)

features = []
for i in range(len(words)):
    features.append([toFloat[i], words[i], posit[i]+0.001, negat[i]+0.001, neutr[i]+0.001, 
                     compo[i]+0.001])

    
# Look at our data
print(label_names)
print('Class label = ', labels[3])
print(feature_names)
print(features[0])

# Split our data
train, test, train_labels, test_labels = train_test_split(features,
                                                          labels,
                                                          test_size=0.25,
                                                          random_state=40)

['Bot' 'Human']
Class label =  Bot
['tweet' 'numwords' 'pos' 'neg' 'neut' 'comp']
[16040, 12, 0.426, 0.001, 0.576, 0.5729]


In [33]:
#This checks to see if the file has been run once

#pickle will help save the classifier state after the classifier has been trained
#pickling is important, as it allows python object hierarchies to turn into bitstreams

#If the file has not been run, run the code below
if run_var == 0:
    classifier = BernoulliNB()
    model = classifier.fit(train, train_labels)
    classify_buffer = open('russian_bots_bernoulli_bayes.pickle', 'wb')
    pickle.dump(model, classify_buffer)
    classify_buffer.close()
    run_var += 1
    

#The alternative is re-opneing the classifier_buffer and updating the classifier    
else:
    classify_buffer = open('russian_bots_bernoulli_bayes.pickle', 'rb')
    try: 
        classifier = pickle.load(classify_buffer)
    except EOFError:
        classify_buffer.close()
        run_var += 1
        pass
        

In [34]:
# Make predictions
preds = classifier.predict(test)
print(preds)
print(len(preds))
counter = 0
for i in range(len(preds)):
    if preds[i] == 'Human':
        counter += 1

print(counter)
# Evaluate accuracy
print(accuracy_score(test_labels, preds))

['Bot' 'Bot' 'Bot' ... 'Bot' 'Bot' 'Bot']
17154
0
0.7218141541331468


In [35]:
print(run_var)

1
